In [ ]:
import pyspark.sql.functions as F
import pyspark as ps
from datetime import date

In [ ]:
# get data

df = spark.sql("""
SELECT *
FROM customer_t
LIMIT 100
""")

check columns and schema

In [ ]:
df.columns

In [ ]:
df.printSchema()

show description table

In [ ]:
# for full table
df.describe().show()

In [ ]:
# for one column
df.describe('customer_id').show()

display result

In [ ]:
# show first 20 rows
df.select('customer_id').distinct().show(20, truncate=False)

In [ ]:
# show first row vertically
df.show(1,vertical=True)

transfer to pandas dataframe

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df1 = df.toPandas()
print(df1)

filter

In [ ]:
df.filter('customer_status=1')

In [ ]:
df.filter(F.col('customer_status')==1)

In [ ]:
df.filter(df.customer_status==1)

In [ ]:
# multiple criteria
de.filter((df['customer_status'].isNotNull()) & ~(df['deceased_customer_flag]))

rename columns

In [ ]:
df2 = df.withColumnRenamed('customer_status', 'customer_status_renamed')

Aggregation

In [ ]:
df.groupby('customer_status').agg(F.countDistinct('customer_id').alias('count'))

In [ ]:
# aggregate on multiple columns
df.groupby('customer_status').agg({'customer_id':'count', 'customer_id':'max'})

In [ ]:
# group by multiple columns
df.groupby('customer_start_datetime', 'customer_status').agg(F.countDistinct('customer_id').alias('count'))

join tables

In [ ]:
df3 = df.join(df2, on=['customer_id'], how='inner')

create new column

In [ ]:
df2 = df.withColumn('new_column', df['customer_id]**2)

In [ ]:
df2 = df.withColumn('new_column', F.when(df.customer_id=1,1).otherwise(0))

sort data

In [ ]:
df.orderby('customer_id')

In [ ]:
df.orderby(df['customer_id'].desc())

In [ ]:
df.sort(F.asc('customer_id'))

handle missing data

In [ ]:
df.na.drop(thresh=2)

In [ ]:
# fill missing data for specific column
df.na.fill(0, subset=['customer_id'])

create pivot table / crosstab

In [ ]:
df.crosstab('customer_status','residence_country_code')

In [ ]:
df.groupby('customer_status').pivot('residence_country_code').agg(F.countDistinct('customer_id').alias('count')).fillna(0)

handle duplicates

In [ ]:
df.dropDuplicates()

create sample dataframe

In [ ]:
df_sample = df.sample(False, 0.2, 42)

# inputs:
# withReplacement: select an observation with or without replacement
# fraction: select x percentage data for the sample
# seed: seed for reproduce the result

UDF (user defined function)

In [ ]:
# create a python function
def lowerCase(str):
    resStr = str.lower()
    return resStr

# convert a python function to pyspark udf
convertUDF = F.udf(lambda x: lowerCase(x))

# use UDF in dataframe
df.convertUDF(F.col('customer_id')).alias('converted_id')

Pearson correlation

In [ ]:
df.select(corr('residence_country_code', 'customer_status'))

save temporary table

In [ ]:
table_name = 'tem_table'
drop_table = f"DROP TABLE IF EXISTS {table_name}"
sqlContext.sql(drop_table)

df.write.mode('overwrite').saveAsTable(table_name)